Image is like OS image - a system that can be replicated. It's layered like onion (Ubuntu image, ROS image, custom layer). We can take an existing docker image, add our docker file that contains extra change to create a new docker image.

Docker image can be run to create a container. We can start/stop container to keep our changes, or run the image again to flash the container.

'docker image ls' or 'docker images' for the list of docker images.
'docker image pull' for downloading docker images.
'docker image rm -f <image-name>' for deleting image.
'docker run ros:humble' for running image.
'docker run -it ros:humble' for interactive in terminal image running.
'docker container ls' or 'docker ps' for list of running containers.
'docker ps -a' for all containers that were ran.
'docker container stop CONTAINER_NAME' or CTRL+D for stopping container.
'docker container start -i CONTAINER_NAME' for starting container.

If we run docker image again, it'll create a fresh container.
'docker container rm CONTAINER_NAME' for deleting CONTAINER_NAME.
'docker container prune' for deleting all containers.

'docker run --rm --name <container_name> <image_name>' will create a container with container_name, and remove it once stopped.

'docker exec -it <container_name> /bin/bash' for opening a new terminal of a running container.
'docker exec -it <container_name> <command_name>' for executing a single command in a new terminal.

Create a folder, open it in VSCODE, add a Dockerfile.
FROM ros:humble #base of the image
RUN will run terminal commands (installing packages)
COPY can copy configs or files in stated directory.

cd to the docker project > 'docker image build -t <image_name> .' the . at the end specifies where I want to build the image, . indicating current directory.

Shared files among many containers: volumes and mounts.

docker run -it -v <absol_path_on_host>:<absol_path_in_cont>

Change directory to my_code, type: 'docker run -it -v $PWD/source:/my_source_code my_image', where $PWD is print working directory/source folder (copy all inside) to /my_source_code directory inside the my_image container.

### ROS Docker image with GUI, adding user

#### Docker file
FROM osrf/ros:humble-desktop-full

- Example of installing programs
RUN apt-get update \
    && apt-get install -y \
    nano \
    vim \
    && rm -rf /var/lib/apt/lists/*

- Example of copying a file
COPY config/ /site_config/

- Create a non-root user
ARG USERNAME=ros
ARG USER_UID=1000
ARG USER_GID=$USER_UID

RUN groupadd --gid $USER_GID $USERNAME \
  && useradd -s /bin/bash --uid $USER_UID --gid $USER_GID -m $USERNAME \
  && mkdir /home/$USERNAME/.config && chown $USER_UID:$USER_GID /home/$USERNAME/.config

- Set up sudo
RUN apt-get update \
  && apt-get install -y sudo \
  && echo $USERNAME ALL=\(root\) NOPASSWD:ALL > /etc/sudoers.d/$USERNAME\
  && chmod 0440 /etc/sudoers.d/$USERNAME \
  && rm -rf /var/lib/apt/lists/*

apt-get is more compitable than apt for automation.
apt-get update is ran everytime we use run for updating package list.
-y adds yes while installing program.
splitting commands into multiple lines with '\': if a new line is added, version control is show that specific change.
We delete package list after running them.

#### Building
sunzid@tuf:~/Desktop/my_project$ docker build -t my_image .

#### Adding user while running docker
sunzid@tuf:~/Desktop/my_code$ docker run -it --user ros -v $PWD/source:/my_source_code my_image

If you add files now, you'll see the user as yourself from host directory. Since you used USER_UID 1000, which is the default USER_UID.

#### Networking while running container
sunzid@tuf:~/Desktop/my_code$ docker run -it --user ros --network=host --ipc=host -v $PWD/source:/my_source_code my_image

docker run -it --user ros --network=host --ipc=host -v $PWD/source:/my_source_code -v /tmp/.X11-unix:/tmp/.X11-unix:rw --env=DISPLAY  my_image

#### Running GUI
xhost +local:

#### Rviz load view
rviz2 -d src/my_bot/config/view_bot.rviz

## Rough

### Turtlebot4

In [ ]:
#  Command to build it
# docker built -t <image name > .
FROM osrf/ros:humble-desktop-full

# Add vscode user with same UID and GID as your host system
# (copied from https://code.visualstudio.com/remote/advancedcontainers/add-nonroot-user#_creating-a-nonroot-user)
ARG USERNAME=vscode
ARG USER_UID=1000
ARG USER_GID=$USER_UID
RUN groupadd --gid $USER_GID $USERNAME \
    && useradd -s /bin/bash --uid $USER_UID --gid $USER_GID -m $USERNAME \
    && apt-get update \
    && apt-get install -y sudo \
    && echo $USERNAME ALL=\(root\) NOPASSWD:ALL > /etc/sudoers.d/$USERNAME \
    && chmod 0440 /etc/sudoers.d/$USERNAME
# Switch from root to user
USER $USERNAME

# Add user to video group to allow access to webcam
RUN sudo usermod --append --groups video $USERNAME

# Update all packages
RUN sudo apt update && sudo apt upgrade -y

# Install Git
RUN sudo apt install -y git && apt-get install -y python3-pip

# Rosdep update (disconnect vpn)
RUN rosdep update

RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-dev-tools

# Install wget
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt-get install -y wget

# Install Ingition
RUN sudo sh -c 'echo "deb http://packages.osrfoundation.org/gazebo/ubuntu-stable `lsb_release -cs` main" > /etc/apt/sources.list.d/gazebo-stable.list'
RUN wget http://packages.osrfoundation.org/gazebo.key -O - | sudo apt-key add -
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt-get install -y ignition-fortress

# Install simulator
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-turtlebot4-simulator

# Add workspace directory
RUN mkdir -p ~/turtlebot4_ws/src && cd ~/turtlebot4_ws/src/

RUN git clone https://github.com/turtlebot/turtlebot4_simulator.git -b humble

RUN cd ..

RUN sudo rosdep init
RUN rosdep update

RUN rosdep install -y --from-path src -yi

# Install catkin_pkg before colcon build
RUN sudo apt update && sudo apt upgrade -y
RUN pip install catkin_pkg

# Source the ROS setup file
RUN echo "source /opt/ros/humble/setup.bash" >> ~/.bashrc

RUN colcon build --symlink-install


RUN echo "ALL Done"

### Turtlebot3

In [ ]:
#  Command to build it
# docker built -t <image name > .
FROM osrf/ros:humble-desktop-full

# Add vscode user with same UID and GID as your host system
# (copied from https://code.visualstudio.com/remote/advancedcontainers/add-nonroot-user#_creating-a-nonroot-user)
ARG USERNAME=vscode
ARG USER_UID=1000
ARG USER_GID=$USER_UID
RUN groupadd --gid $USER_GID $USERNAME \
    && useradd -s /bin/bash --uid $USER_UID --gid $USER_GID -m $USERNAME \
    && apt-get update \
    && apt-get install -y sudo \
    && echo $USERNAME ALL=\(root\) NOPASSWD:ALL > /etc/sudoers.d/$USERNAME \
    && chmod 0440 /etc/sudoers.d/$USERNAME
# Switch from root to user
USER $USERNAME

# Add user to video group to allow access to webcam
RUN sudo usermod --append --groups video $USERNAME

# Update all packages
RUN sudo apt update && sudo apt upgrade -y

# Install Git
RUN sudo apt install -y git

RUN sudo apt install -y python3

# Rosdep update
RUN rosdep update

RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-dev-tools

# Install wget
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt-get install -y wget

############################################################
## Turtlebot3 Simulation
# Install Ingition
RUN sudo sh -c 'echo "deb http://packages.osrfoundation.org/gazebo/ubuntu-stable `lsb_release -cs` main" > /etc/apt/sources.list.d/gazebo-stable.list'
RUN wget http://packages.osrfoundation.org/gazebo.key -O - | sudo apt-key add -
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt-get install -y ignition-fortress

# Install simulator
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-turtlebot4-simulator

# Add workspace directory
RUN mkdir -p ~/turtlebot4_ws/src && cd ~/turtlebot4_ws/src/

RUN git clone https://github.com/turtlebot/turtlebot4_simulator.git -b humble

RUN cd ..

RUN sudo rosdep init
RUN rosdep update

RUN rosdep install -y --from-path src -yi

# Install catkin_pkg before colcon build
RUN sudo apt update && sudo apt upgrade -y
RUN pip install catkin_pkg

# Source the ROS setup file
RUN echo "source /opt/ros/humble/setup.bash" >> ~/.bashrc

RUN colcon build --symlink-install


############################################################
## Turtlebot3 Simulation

# Gazebo install
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-gazebo-*

# Cartographer
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-cartographer
RUN sudo apt install -y ros-humble-cartographer-ros

# Navigation 2
RUN sudo apt update && sudo apt upgrade -y
RUN sudo apt install -y ros-humble-navigation2
RUN sudo apt install -y ros-humble-nav2-bringup

# Turtlebot3 packages
RUN sudo apt update && sudo apt upgrade -y
RUN source ~/.bashrc
RUN sudo apt install -y ros-humble-dynamixel-sdk
RUN sudo apt install -y ros-humble-turtlebot3-msgs
RUN sudo apt install -y ros-humble-turtlebot3

# Environment configuration
RUN sudo apt update && sudo apt upgrade -y
RUN echo 'export ROS_DOMAIN_ID=30 #TURTLEBOT3' >> ~/.bashrc
RUN source ~/.bashrc

# Turtlebot3 simulation
RUN cd ~/turtlebot3/src
RUN git clone -b humble-devel https://github.com/ROBOTIS-GIT/turtlebot3_simulations.git
RUN cd ~/turtlebot3_ws && colcon build --symlink-install


RUN echo "ALL Done"

Build the dockerfile and get Docker image ID: docker build -t r2_humble_turtlebot34 .

make the bash file and Dockerfile executable

./run_docker_gpu.bash

In [ ]:
xhost local:root

XAUTH=/tmp/.docker.xauth

docker run -it \
    --name=turtlebot4_simulator_container \
    --env="DISPLAY=$DISPLAY" \
    --env="QT_X11_NO_MITSHM=1" \
    --volume="/tmp/.X11-unix:/tmp/.X11-unix:rw" \
    --env="XAUTHORITY=$XAUTH" \
    --volume="$XAUTH:$XAUTH" \
    --net=host \
    --privileged \
    --runtime=nvidia \
    r2_humble_turtlebot34 \
    bash

echo "Done."